In [1]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
import glove
from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D

from keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
import np_utils
from tensorflow.python.keras.backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

2023-11-23 14:24:05.647446: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 14:24:05.647478: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-23 14:24:05.647503: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-23 14:24:05.653333: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def reset_keras(model):
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted
def create_dataset(dict_of_ner):
    temp_data = []
    for k, v in sorted(dict_of_ner.items()):
        temp = []
        for embed in v:
            temp.append(embed)
        temp_data.append(np.mean(temp, axis = 0)) 
    return np.asarray(temp_data)

def make_prediction_multi_avg(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_multi_avg(predictions, probs, ground_truth, 
                          
                          embed_name, problem_type, iteration, hidden_unit_size,
                          
                          sequence_name, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    result_dict = {}    
    result_dict['iteration'] = [iteration]
    result_dict['auc'] = [auc]
    result_dict['auprc'] = [auprc]
    result_dict['acc'] = [acc]
    result_dict['F1'] = [F1]
        
    filename = str(sequence_name)+"-"+str(hidden_unit_size)+"-"+embed_name+"-"+problem_type+"-"+type_of_ner+"-avg.csv"
    
    result_path = "../results/multimodal-baseline/metrics/"+filename
    df = pd.DataFrame.from_dict(result_dict)
    df.to_csv(result_path,mode='a',index=False,header=False)

    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1

    result_path = "../results/multimodal-baseline/metrics/"
    file_name = str(sequence_name)+"-"+str(hidden_unit_size)+"-"+embed_name
    file_name = file_name +"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+"-avg-.p"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))
    print(auc, auprc, acc, F1)
    
def avg_ner_model(layer_name, number_of_unit, embedding_name):

    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100

    sequence_input = Input(shape=(24,104))

    input_avg = Input(shape=(input_dimension, ), name = "avg")        
#     x_1 = Dense(256, activation='relu')(input_avg)
#     x_1 = Dropout(0.3)(x_1)
    
    if layer_name == "GRU":
        x = GRU(number_of_unit)(sequence_input)
    elif layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)

    x = keras.layers.Concatenate()([x, input_avg])

    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    
    logits_regularizer = regularizers.l2(l2=0.01)
    
    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.GlorotNormal(), 
                  kernel_regularizer=logits_regularizer)(x)
    
    
    opt = Adam(learning_rate=0.001, weight_decay=0.01)
    model = Model(inputs=[sequence_input, input_avg], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['acc'])
    
    return model

In [3]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("../data/timeseries/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("../data/timeseries/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("../data/timeseries/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("../data/timeseries/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("../data/timeseries/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("../data/timeseries/"+type_of_ner+"_y_test.pkl")

ner_word2vec = pd.read_pickle("../data/clinical_notes/representations/"+type_of_ner+"_ner_word2vec_limited_dict.pkl")
ner_fasttext = pd.read_pickle("../data/clinical_notes/representations/"+type_of_ner+"_ner_fasttext_limited_dict.pkl")

ner_concat = pd.read_pickle("../data/clinical_notes/representations/"+type_of_ner+"_ner_combined_limited_dict.pkl")

train_ids = pd.read_pickle("../data/timeseries/"+type_of_ner+"_train_ids.pkl")
dev_ids = pd.read_pickle("../data/timeseries/"+type_of_ner+"_dev_ids.pkl")
test_ids = pd.read_pickle("../data/timeseries/"+type_of_ner+"_test_ids.pkl")

In [6]:
embedding_types = ['word2vec','fasttext','concat']
embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']


num_epoch = 100
model_patience = 5
monitor_criteria = 'val_loss'
batch_size = 64
iter_num = 2
unit_sizes = [128, 256]

#layers = ["LSTM", "GRU"]
layers = ["GRU"]
for each_layer in layers:
    print ("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print ("Hidden unit: ", each_unit_size)

        for embed_dict, embed_name in zip(embedding_dict, embedding_types):    
            print ("Embedding: ", embed_name)
            print("=============================")

            temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
            temp_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
            temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

            x_train_ner = create_dataset(temp_train_ner)
            x_dev_ner = create_dataset(temp_dev_ner)
            x_test_ner = create_dataset(temp_test_ner)


            for iteration in range(1, iter_num):
                print ("Iteration number: ", iteration)

                for each_problem in target_problems:
                    print ("Problem type: ", each_problem)
                    print ("__________________")

                    early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                    best_model_name = "../results/multimodal-baseline/best_models/"+"avg-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"
                    checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                        save_best_only=True, mode='min', save_freq='epoch')
                    #checkpoint._supports_tf_logs = False



                    callbacks = [early_stopping_monitor, checkpoint]

                    model = avg_ner_model(each_layer, each_unit_size, embed_name)
                    
                    model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1, 
                              validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, 
                              batch_size=batch_size )

                    model.load_weights(best_model_name)

                    probs, predictions = make_prediction_multi_avg(model, [x_test_lstm, x_test_ner])
                    
                    save_scores_multi_avg(predictions, probs, y_test[each_problem], 
                                embed_name, each_problem, iteration, each_unit_size, 
                                each_layer, type_of_ner)
                    
                    reset_keras(model)
                    #del model
                    clear_session()
                    gc.collect()

Layer:  GRU
Hidden unit:  128
Embedding:  fasttext2
Iteration number:  1
Problem type:  mort_hosp
__________________
Epoch 1/100
226/242 [===========================>..] - ETA: 0s - loss: 0.2802 - acc: 0.9054
Epoch 1: val_loss improved from inf to 0.23903, saving model to ../results/multimodal-baseline/best_models/avg-fasttext2-mort_hosp-best_model.hdf5
242/242 [==============================] - 2s 4ms/step - loss: 0.2774 - acc: 0.9059 - val_loss: 0.2390 - val_acc: 0.9133
Epoch 2/100
 52/242 [=====>........................] - ETA: 0s - loss: 0.2482 - acc: 0.9102

/data/apps/miniforge3/envs/cse6250-project/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/242 [============================>.] - ETA: 0s - loss: 0.2391 - acc: 0.9147
Epoch 2: val_loss improved from 0.23903 to 0.23093, saving model to ../results/multimodal-baseline/best_models/avg-fasttext2-mort_hosp-best_model.hdf5
242/242 [==============================] - 1s 4ms/step - loss: 0.2387 - acc: 0.9147 - val_loss: 0.2309 - val_acc: 0.9160
Epoch 3/100
233/242 [===========================>..] - ETA: 0s - loss: 0.2228 - acc: 0.9183
Epoch 3: val_loss did not improve from 0.23093
242/242 [==============================] - 1s 4ms/step - loss: 0.2246 - acc: 0.9177 - val_loss: 0.2336 - val_acc: 0.9114
Epoch 4/100
238/242 [============================>.] - ETA: 0s - loss: 0.2122 - acc: 0.9233
Epoch 4: val_loss improved from 0.23093 to 0.22848, saving model to ../results/multimodal-baseline/best_models/avg-fasttext2-mort_hosp-best_model.hdf5
242/242 [==============================] - 1s 3ms/step - loss: 0.2121 - acc: 0.9232 - val_loss: 0.2285 - val_acc: 0.9183
Epoch 5/100
228/242 [===

2023-11-23 15:31:53.100328: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14529 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:2b:00.0, compute capability: 7.5
2023-11-23 15:31:53.110168: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14529 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:2b:00.0, compute capability: 7.5


Problem type:  mort_icu
__________________
Epoch 1/100
232/242 [===========================>..] - ETA: 0s - loss: 0.2115 - acc: 0.9327
Epoch 1: val_loss improved from inf to 0.17445, saving model to ../results/multimodal-baseline/best_models/avg-fasttext2-mort_icu-best_model.hdf5
242/242 [==============================] - 2s 4ms/step - loss: 0.2111 - acc: 0.9327 - val_loss: 0.1745 - val_acc: 0.9419
Epoch 2/100
 60/242 [======>.......................] - ETA: 0s - loss: 0.1571 - acc: 0.9474

...

Problem type:  los_7
__________________
Epoch 1/100
231/242 [===========================>..] - ETA: 0s - loss: 0.2864 - acc: 0.9186
Epoch 1: val_loss improved from inf to 0.26640, saving model to ../results/multimodal-baseline/best_models/avg-concat2-los_7-best_model.hdf5
242/242 [==============================] - 2s 5ms/step - loss: 0.2854 - acc: 0.9188 - val_loss: 0.2664 - val_acc: 0.9233
Epoch 2/100
 47/242 [====>.........................] - ETA: 0s - loss: 0.2410 - acc: 0.9259

/data/apps/miniforge3/envs/cse6250-project/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


230/242 [===========================>..] - ETA: 0s - loss: 0.2559 - acc: 0.9216
Epoch 2: val_loss improved from 0.26640 to 0.26111, saving model to ../results/multimodal-baseline/best_models/avg-concat2-los_7-best_model.hdf5
242/242 [==============================] - 1s 4ms/step - loss: 0.2549 - acc: 0.9220 - val_loss: 0.2611 - val_acc: 0.9233
Epoch 3/100
236/242 [============================>.] - ETA: 0s - loss: 0.2434 - acc: 0.9219
Epoch 3: val_loss improved from 0.26111 to 0.25873, saving model to ../results/multimodal-baseline/best_models/avg-concat2-los_7-best_model.hdf5
242/242 [==============================] - 1s 4ms/step - loss: 0.2432 - acc: 0.9222 - val_loss: 0.2587 - val_acc: 0.9233
Epoch 4/100
237/242 [============================>.] - ETA: 0s - loss: 0.2349 - acc: 0.9227
Epoch 4: val_loss did not improve from 0.25873
242/242 [==============================] - 1s 4ms/step - loss: 0.2353 - acc: 0.9226 - val_loss: 0.2592 - val_acc: 0.9233
Epoch 5/100
236/242 [===============

2023-11-23 15:34:37.810469: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14529 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:2b:00.0, compute capability: 7.5
2023-11-23 15:34:37.819728: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14529 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:2b:00.0, compute capability: 7.5


: 